# Análisis de Texto con spaCy - Google Forms

Este notebook demuestra cómo analizar respuestas de texto de Google Forms usando la biblioteca spaCy.

## 1. Configuración Inicial

Importar las bibliotecas necesarias y configurar el entorno.

In [ ]:
import sys
import os

# Agregar el directorio raíz al path
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import matplotlib.pyplot as plt
from src.data_loader import load_data, preprocess_dataframe
from src.text_analyzer import TextAnalyzer
from src.visualizer import *
import config

%matplotlib inline
print("Bibliotecas importadas correctamente")

## 2. Cargar Datos

Cargar el archivo CSV de Google Forms.

In [ ]:
# Ruta al archivo de datos
data_file = "../data/ejemplo_formulario.csv"

# Cargar datos
df = load_data(data_file)

print(f"Datos cargados: {len(df)} filas")
print(f"Columnas: {df.columns.tolist()}")
df.head()

## 3. Preprocesar Datos

Limpiar y preparar los datos para el análisis.

In [ ]:
# Preprocesar
df_clean = preprocess_dataframe(df, text_column='Respuesta')

print(f"Datos limpios: {len(df_clean)} filas")
df_clean.head()

## 4. Inicializar Analizador spaCy

Crear una instancia del analizador de texto.

In [ ]:
# Inicializar analizador con modelo en español
analyzer = TextAnalyzer(model_name='es_core_news_sm')
print("Analizador inicializado")

## 5. Analizar un Texto de Ejemplo

Veamos un análisis detallado de un solo texto.

In [ ]:
# Analizar el primer texto
sample_text = df_clean['Respuesta'].iloc[0]
result = analyzer.analyze_text(sample_text)

print("Texto original:")
print(sample_text)
print("\nAnálisis:")
print(f"Número de tokens: {result['num_tokens']}")
print(f"Número de oraciones: {result['num_sentences']}")
print(f"\nEntidades encontradas: {result['entities']}")
print(f"\nSustantivos: {result['nouns'][:10]}")
print(f"Verbos: {result['verbs'][:10]}")
print(f"Adjetivos: {result['adjectives'][:10]}")

## 6. Análisis de Todos los Textos

Analizar todas las respuestas del formulario.

In [ ]:
# Obtener lista de textos
texts = df_clean['Respuesta'].tolist()

# Obtener estadísticas
stats_df = analyzer.get_sentiment_statistics(texts)
stats_df.head()

In [ ]:
# Estadísticas descriptivas
print("Estadísticas generales:")
print(stats_df.describe())

## 7. Palabras Más Frecuentes

Identificar las palabras más comunes en las respuestas.

In [ ]:
# Top 30 palabras más frecuentes
top_words = analyzer.get_top_words(texts, n=30)

# Mostrar como DataFrame
words_df = pd.DataFrame(top_words, columns=['Palabra', 'Frecuencia'])
print("Palabras más frecuentes:")
words_df.head(20)

In [ ]:
# Visualizar frecuencia de palabras
fig = plot_word_frequency(top_words, "Palabras Más Frecuentes", top_n=20)
plt.show()

In [ ]:
# Crear nube de palabras
fig = create_wordcloud(top_words, "Nube de Palabras")
plt.show()

## 8. Análisis por Categoría Gramatical

Analizar sustantivos, verbos y adjetivos por separado.

In [ ]:
# Top sustantivos
top_nouns = analyzer.get_top_words(texts, n=20, pos_filter=['NOUN'])
nouns_df = pd.DataFrame(top_nouns, columns=['Sustantivo', 'Frecuencia'])
print("Sustantivos más frecuentes:")
print(nouns_df)

In [ ]:
# Top verbos
top_verbs = analyzer.get_top_words(texts, n=20, pos_filter=['VERB'])
verbs_df = pd.DataFrame(top_verbs, columns=['Verbo', 'Frecuencia'])
print("Verbos más frecuentes:")
print(verbs_df)

In [ ]:
# Top adjetivos
top_adjectives = analyzer.get_top_words(texts, n=20, pos_filter=['ADJ'])
adjectives_df = pd.DataFrame(top_adjectives, columns=['Adjetivo', 'Frecuencia'])
print("Adjetivos más frecuentes:")
print(adjectives_df)

## 9. Extracción de Entidades Nombradas

Identificar entidades como personas, lugares, organizaciones, etc.

In [ ]:
# Extraer entidades
entities_df = analyzer.extract_entities(texts)

if not entities_df.empty:
    print(f"Total de entidades encontradas: {len(entities_df)}")
    print(f"\nTipos de entidades: {entities_df['label'].unique().tolist()}")
    print("\nPrimeras entidades:")
    print(entities_df.head(20))
else:
    print("No se encontraron entidades nombradas")

In [ ]:
# Distribución de tipos de entidades
if not entities_df.empty:
    entity_type_counts = entities_df['label'].value_counts()
    print("Distribución por tipo de entidad:")
    print(entity_type_counts)
    
    # Visualizar
    fig = plot_entity_distribution(entities_df, "Distribución de Entidades Nombradas")
    if fig:
        plt.show()

## 10. Visualización de Estadísticas

Crear visualizaciones de las estadísticas del texto.

In [ ]:
# Panel múltiple de estadísticas
fig = plot_multiple_statistics(stats_df)
plt.show()

## 11. Exportar Resultados

Guardar los resultados del análisis.

In [ ]:
from src.data_loader import save_results

# Asegurar que existe el directorio de salida
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

# Guardar estadísticas
stats_combined = pd.concat([df_clean.reset_index(drop=True), stats_df], axis=1)
save_results(stats_combined, 'notebook_statistics', format='csv')

# Guardar palabras frecuentes
save_results(words_df, 'notebook_top_words', format='csv')

# Guardar entidades si existen
if not entities_df.empty:
    save_results(entities_df, 'notebook_entities', format='csv')

print("Resultados exportados exitosamente")

## Conclusiones

Este notebook ha demostrado cómo:
1. Cargar datos de Google Forms
2. Preprocesar texto
3. Realizar análisis NLP con spaCy
4. Extraer entidades nombradas
5. Visualizar resultados
6. Exportar análisis

Puedes adaptar este notebook a tus propios datos modificando la ruta del archivo y los nombres de las columnas.